In [1]:
import os
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OrdinalEncoder,StandardScaler

def freedman(data):
    # freedman method for choosing no of bins
    data = np.asarray(data, dtype=np.float_)
    IQR = scipy.stats.iqr(data, rng=(25, 75), scale=1, nan_policy="omit")
    N = data.size
    bw = (2 * IQR) / np.power(N, 1/3)
    mini= np.min(data)
    maxi=np.max(data)
    d = maxi-mini
    result = int((d / bw) + 1)
    return result

In [2]:
# loading all csv files using pandas
PATH = '../datasets/'
Study_A = pd.read_csv(PATH+"Study_A.csv")
Study_B = pd.read_csv(PATH+"Study_B.csv")
Study_C = pd.read_csv(PATH+"Study_C.csv")
Study_D = pd.read_csv(PATH+"Study_D.csv")
Study_E = pd.read_csv(PATH+"Study_E.csv") 

In [3]:
all_studies = pd.concat([Study_A,Study_B,Study_C,Study_D,Study_E]) 
# considering all given studies as we have data of PANSS_Score and visit days for all studies 

In [4]:
all_studies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22909 entries, 0 to 1961
Data columns (total 40 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Study         22909 non-null  object
 1   Country       22909 non-null  object
 2   PatientID     22909 non-null  int64 
 3   SiteID        22909 non-null  int64 
 4   RaterID       22909 non-null  int64 
 5   AssessmentiD  22909 non-null  int64 
 6   TxGroup       22909 non-null  object
 7   VisitDay      22909 non-null  int64 
 8   P1            22909 non-null  int64 
 9   P2            22909 non-null  int64 
 10  P3            22909 non-null  int64 
 11  P4            22909 non-null  int64 
 12  P5            22909 non-null  int64 
 13  P6            22909 non-null  int64 
 14  P7            22909 non-null  int64 
 15  N1            22909 non-null  int64 
 16  N2            22909 non-null  int64 
 17  N3            22909 non-null  int64 
 18  N4            22909 non-null  int64 
 19  N5   

In [5]:
all_studies["P_total"] = np.sum(np.array([all_studies[f"P{i}"] for i in range(1,8)]),axis=0)
all_studies["N_total"] = np.sum(np.array([all_studies[f"N{i}"] for i in range(1,8)]),axis=0)
all_studies["G_total"] = np.sum(np.array([all_studies[f"G{i}"] for i in range(1,17)]),axis=0)

In [6]:
all_studies_control = all_studies[all_studies["TxGroup"]=="Control"]
all_studies_treatment = all_studies[all_studies["TxGroup"]!="Control"]
# grouping all studies A,B...E depending on the treatment or control groups

In [7]:
ordinal_encoder = OrdinalEncoder()
arr = np.array(all_studies["Country"]).reshape(-1,1)
arr1 = np.array(all_studies["TxGroup"]).reshape(-1,1)
arr2 = np.array(all_studies["Study"]).reshape(-1,1)
encoded = ordinal_encoder.fit_transform(arr)
encoded1 = ordinal_encoder.fit_transform(arr1)
encoded2 = ordinal_encoder.fit_transform(arr2)
all_studies.loc[:,"Country"] = encoded
all_studies.loc[:,"TxGroup"] = encoded1
all_studies.loc[:,"Study"] = encoded2
# encoding all text categorial attributes into numbers using OrdinalEncoder from scikit-learn

C:\Users\Hamesh\AppData\Local\Temp\ipykernel_18948\1305227138.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_studies.loc[:,"Country"] = encoded
C:\Users\Hamesh\AppData\Local\Temp\ipykernel_18948\1305227138.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_studies.loc[:,"TxGroup"] = encoded1
C:\Users\Hamesh\AppData\Local\Temp\ipykernel_18948\1305227138.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavio

In [8]:
all_studies.describe()

,Study,Country,PatientID,SiteID,RaterID,AssessmentiD,TxGroup,VisitDay,P1,P2,...,G11,G12,G13,G14,G15,G16,PANSS_Total,P_total,N_total,G_total
count,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,...,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000,22909.000000
mean,2.007901,16.807630,30633.481208,60224.140993,90382.462526,305387.675062,0.496704,89.957571,2.958663,2.673054,...,2.386049,3.207735,2.459252,1.803047,2.426383,2.652538,71.112096,16.396962,19.848793,34.866341
std,0.986454,8.473701,9883.579593,19728.216854,29596.641078,98758.258590,0.500000,92.961899,1.404361,1.235907,...,1.033906,1.211673,1.058141,0.972589,1.116864,1.185041,18.908947,6.472019,5.520974,9.586275
min,0.000000,0.000000,10001.000000,20001.000000,30001.000000,100001.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,7.000000,7.000000,16.000000
25%,2.000000,7.000000,30163.000000,60016.000000,90039.000000,301440.000000,0.000000,15.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,58.000000,11.000000,16.000000,28.000000
50%,2.000000,18.000000,30865.000000,60101.000000,90230.000000,307167.000000,0.000000,67.000000,3.000000,3.000000,...,2.000000,3.000000,3.000000,1.000000,2.000000,3.000000,69.000000,15.000000,20.000000,34.000000
75%,2.000000,27.000000,31529.000000,60187.000000,90407.000000,312894.000000,1.000000,129.000000,4.000000,3.000000,...,3.000000,4.000000,3.000000,3.000000,3.000000,3.000000,84.000000,21.000000,23.000000,41.000000
max,4.000000,28.000000,50513.000000,100061.000000,150139.000000,502370.000000,1.000000,480.000000,7.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,166.000000,43.000000,46.000000,83.000000


In [9]:
Study_A_grouped = []
same_id = Study_A.iloc[0,:]["PatientID"] # initialized id
dummy = []

dummy.append(Study_A.iloc[0,:])

for index,row in Study_A.iloc[1:,:].iterrows():
    if(row["PatientID"]!=same_id):
        Study_A_grouped.append(dummy)
        dummy=[]
        dummy.append(row)
        same_id = row["PatientID"]
    else:
        dummy.append(row)

### PATIENT SEGMENTATION

In [10]:
all_initial = all_studies[all_studies["VisitDay"] == 0]

In [11]:
subset_data = all_initial.iloc(["P_total","N_total","G_total","SiteID","AssessmentiD","PatientID","LeadStatus"],axis=1)

In [12]:
scaler = StandardScaler()
scaler.fit(subset_data)
subset_data = scaler.transform(subset_data)

#### USING GMM

In [13]:
maxrangeN=14 #denotes x which is from 1 to x we check the corresponding paramter models score and test to find optimum N
models = [] #Gaussian mixture of all possible number of components from 1 to max range N
for i in range(maxrangeN):
    models.append(GaussianMixture(n_components=i+1))
    models[i].fit(subset_data) #fitting model with i number of components on 
AIC = [m.aic(subset_data) for m in models] # Computing AIC for those models
BIC = [m.bic(subset_data) for m in models] # Computing BIC for those models
index = np.argmin(BIC) # computes index of the model with least BIC
optimum_n_model = models[index]
print("best fit converged:", optimum_n_model.converged_)
print("For Least BIC we have optimum number of components as",index+1)

best fit converged: True
For Least BIC we have optimum number of components as 7
